In [ ]:
!pip install -qqq openai==0.28

In [ ]:
# create input questions file if not provided, otherwise don't run this cell
with open('input_questions.txt', 'w') as file:
    file.write('''Show me the top 5 highest-paid employees.
Who is the youngest employee in the Engineering department?
Find all employees who were hired in the year 2019.
Show the names of employees whose salary is above the average salary in their department.
Display the names of employees whose first and last names start with the same letter.
Find the employee with the highest salary in each department
''')

**Database Creation**

In [27]:
# creating database if not provided, otherwise don't run this cell

import sqlite3

def create_database():
    # Connect to SQLite database with a timeout
    conn = sqlite3.connect('database.db', timeout=10)  # Timeout in seconds
    cursor = conn.cursor()

    # Set journal mode to WAL for better concurrency handling
    cursor.execute("PRAGMA journal_mode=WAL;")

    # Drop the existing table if it exists
    cursor.execute('''
    DROP TABLE IF EXISTS employees;
    ''')

    # Create a new table
    cursor.execute('''
    CREATE TABLE employees (
        EmployeeID INTEGER PRIMARY KEY,
        FirstName TEXT,
        LastName TEXT,
        Age INTEGER,
        Department TEXT,
        Position TEXT,
        Salary INTEGER,
        HireDate TEXT,
        ManagerID INTEGER
    )
    ''')

    # Data to insert into the employees table
    employees_data = [
        (1, "John", "Smith", 28, "Sales", "Manager", 85000, "2015-03-01", 21),
        (2, "Jane", "Doe", 34, "Engineering", "Developer", 95000, "2016-05-23", 6),
        (3, "Emily", "Johnson", 29, "HR", "Generalist", 70000, "2017-08-14", 7),
        (4, "Michael", "Brown", 45, "Marketing", "Coordinator", 80000, "2014-11-11", 20),
        (5, "Sarah", "Williams", 31, "Sales", "Consultant", 85000, "2013-07-25", 1),
        (6, "David", "Jones", 38, "Engineering", "Manager", 95000, "2019-09-17", 1),
        (7, "Laura", "Garcia", 26, "HR", "Manager", 70000, "2018-12-02", 21),
        (8, "James", "Miller", 39, "Marketing", "SEO", 80000, "2020-01-10", 20),
        (9, "Anna", "Davis", 27, "Sales", "Associate", 85000, "2015-03-01", 1),
        (10, "Robert", "Rodriguez", 41, "Engineering", "QA", 95000, "2016-05-23", 6),
        (11, "Linda", "Martinez", 33, "HR", "Coordinator", 70000, "2017-08-14", 7),
        (12, "William", "Hernandez", 30, "Marketing", "Analyst", 80000, "2014-11-11", 20),
        (13, "Elizabeth", "Lopez", 36, "Sales", "Analyst", 85000, "2013-07-25", 1),
        (14, "Richard", "Gonzalez", 42, "Engineering", "DevOps", 95000, "2019-09-17", 6),
        (15, "Jessica", "Wilson", 32, "HR", "Analyst", 70000, "2018-12-02", 7),
        (16, "Joseph", "Anderson", 37, "Marketing", "Associate", 80000, "2020-01-10", 20),
        (17, "Karen", "Thomas", 29, "Sales", "Coordinator", 85000, "2015-03-01", 1),
        (18, "Thomas", "Taylor", 35, "Engineering", "Technical support", 95000, "2016-05-23", 6),
        (19, "Nancy", "Moore", 40, "HR", "Recruiter", 70000, "2017-08-14", 7),
        (20, "Charles", "Jackson", 43, "Marketing", "Manager", 80000, "2014-11-11", 21),
        (21, "Alex", "Johnson", 50, "Management", "CEO", 200000, "2010-01-01", None)
    ]

    # Insert data into the employees table
    cursor.executemany('''
    INSERT INTO employees (EmployeeID, FirstName, LastName, Age, Department, Position, Salary, HireDate, ManagerID)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', employees_data)

    # -------------------------------Uncomment this to check for multitable and joins---------------------------
    # # create another table for handling multitable and joins
    # cursor.execute('''
    # DROP TABLE IF EXISTS departments;
    # ''')

    # cursor.execute('''
    # CREATE TABLE departments (
    #     DepartmentID INTEGER PRIMARY KEY,
    #     DepartmentName TEXT
    # )
    # ''')

    # departments_data = [
    #     (1, "Sales"),
    #     (2, "Engineering"),
    #     (3, "HR"),
    #     (4, "Marketing"),
    #     (5, "Management")
    # ]

    # # Insert data into the departments table
    # cursor.executemany('''
    # INSERT INTO departments (DepartmentID, DepartmentName)
    # VALUES (?, ?)
    # ''', departments_data)

    # Commit the changes and close the connection
    conn.commit()
    conn.close()

    print("Database created and populated successfully.")

if __name__ == "__main__":
    create_database()


Database created and populated successfully.


In [26]:
import openai
import sqlparse
import sqlite3
import re

# Set your OpenAI API key here
openai.api_key = '<Add your API Key>'

def connect_database(db_file='database.db'):
    """Connect to the SQLite database."""
    return sqlite3.connect(db_file)

def execute_query(db_connection, query):
    """Execute a SQL query on the given database and return the results."""
    cursor = db_connection.cursor()
    try:
        cursor.execute(query)
        results = cursor.fetchall()
        return results
    except sqlite3.OperationalError as e:
        return f"An error occurred: {e}"

def read_input(file_path):
    """Read natural language questions from a file."""
    with open(file_path, 'r') as file:
        questions = file.readlines()
    return [q.strip() for q in questions]

def preprocess_query(sql_query):
    """Preprocess the SQL query to ensure proper handling of dates in DD-MM-YYYY format."""
    sql_query = re.sub(r"YEAR\((\w+)\)", r"substr(\1, 7, 4)", sql_query)
    sql_query = re.sub(r"(\w+)\s*>=\s*'(\d{4})-01-01'\s*AND\s*\1\s*<\s*'(\d{4})-01-01'",
                       r"substr(\1, 7, 4) = '\2'", sql_query)
    return sql_query

def generate_sql_query(question, schema_info):
    """Generate an SQL query from a natural language question, including schema info."""
    hint = "Use SQLite-compatible syntax and avoid unsupported functions like STR_TO_DATE."
    prompt = f"{schema_info}\n\n{hint}\n\nConvert the following natural language question into an SQL query:\n\nQuestion: {question}\nSQL Query:"
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=150
    )
    sql_query = response['choices'][0]['message']['content'].strip()
    formatted_sql = sqlparse.format(sql_query, reindent=True, keyword_case='upper')
    preprocessed_sql = preprocess_query(formatted_sql)
    return preprocessed_sql

def optimize_sql_query(sql_query):
    """Generate optimization suggestions for an SQL query using OpenAI's model."""
    prompt = f"Here is an SQL query:\n\n{sql_query}\n\nSuggest optimizations to improve the performance of this SQL query:"
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=150
    )
    optimization_suggestions = response['choices'][0]['message']['content'].strip()
    formatted_optimizations = sqlparse.format(optimization_suggestions, reindent=True, keyword_case='upper')
    return formatted_optimizations

def save_output_to_file(file_number, question, sql_query, results, suggestions):
    """Save the output to a text file."""
    output_file = f"question_{file_number}.txt"
    with open(output_file, 'w') as file:
        file.write("========================================\n")
        file.write(f"              Question {file_number}              \n")
        file.write("========================================\n\n")
        file.write(f"Question:\n{question}\n\n")
        file.write("Generated SQL Query:\n")
        file.write(f"{sql_query}\n\n")
        file.write("Query Results:\n")
        if isinstance(results, list):
            if results:
                for row in results:
                    file.write(f"{row}\n")
            else:
                file.write("No results found.\n")
        else:
            file.write(f"{results}\n")
        file.write("\nOptimization Suggestions:\n")
        formatted_suggestions = format_optimization_suggestions(suggestions)
        file.write(f"{formatted_suggestions}\n")
        file.write("\n" + "="*40 + "\n")

def format_optimization_suggestions(suggestions):
    """Format optimization suggestions for better readability."""
    lines = suggestions.split("\n")
    formatted_lines = []
    for line in lines:
        line = line.strip()
        if line.startswith("1.") or line.startswith("2.") or line.startswith("3.") or line.startswith("4.") or line.startswith("5."):
            formatted_lines.append(f"\n{line}")
        else:
            formatted_lines.append(f"   {line}")
    return "\n".join(formatted_lines)

def main():
    schema_info = """
    The database consists of one table named 'employees' with the following columns:
    - EmployeeID (integer)
    - FirstName (text)
    - LastName (text)
    - Age (integer)
    - Department (text)
    - Position (text)
    - Salary (numeric)
    - HireDate (date in DD-MM-YYYY format)
    - ManagerID (integer, optional)
    """
    input_file = 'input_questions.txt'
    questions = read_input(input_file)
    db_connection = connect_database()

    for i, question in enumerate(questions, start=1):
        print(f"\nProcessing Question {i}: {question}\n")
        sql_query = generate_sql_query(question, schema_info)
        print("Generated SQL Query:")
        print(sql_query)

        # Execute the SQL query in the database and display results
        results = execute_query(db_connection, sql_query)
        print("Query Results:")
        if isinstance(results, list):
            for row in results:
                print(row)
        else:
            print(results)

        # Generate and print optimization suggestions
        suggestions = optimize_sql_query(sql_query)
        print("\nOptimization Suggestions:")
        print(suggestions)

        # Save output to a file
        save_output_to_file(i, question, sql_query, results, suggestions)

        print("-" * 80)

    db_connection.close()

if __name__ == "__main__":
    main()



Processing Question 1: Show me the top 5 highest-paid employees.

Generated SQL Query:
SELECT *
FROM employees
ORDER BY Salary DESC
LIMIT 5;
Query Results:
(21, 'Alex', 'Johnson', 50, 'Management', 'CEO', 200000, '2010-01-01', None)
(2, 'Jane', 'Doe', 34, 'Engineering', 'Developer', 95000, '2016-05-23', 6)
(6, 'David', 'Jones', 38, 'Engineering', 'Manager', 95000, '2019-09-17', 1)
(10, 'Robert', 'Rodriguez', 41, 'Engineering', 'QA', 95000, '2016-05-23', 6)
(14, 'Richard', 'Gonzalez', 42, 'Engineering', 'DevOps', 95000, '2019-09-17', 6)

Optimization Suggestions:
1. ADD an INDEX ON the `Salary` COLUMN IN the `employees` table. This will speed up the sorting process WHEN the query IS executed. 2. Consider limiting the number OF columns selected IN the query INSTEAD OF USING `SELECT *`. ONLY
SELECT the columns that ARE necessary
FOR the query TO REDUCE the amount OF DATA being processed. 3. IF the `Salary` COLUMN IS frequently used
FOR sorting,
    consider denormalizing the DATA BY stor